# Inverted Pendulum DQN - Grid Search

This script is an extension to the inverted pendulum DQN algorithm in inv_pend.ipynb. This scripts automates the grid search in a hyperparameter space to explore the best performance of DQN.

TO DO:
+ Package functions related to the experiment in a `GridSearch` class
    - **Attributes:**
        - Hyperparameters - model arch, lr , buffer_size, min_replay_szie, target_update_freq, gamma, eps_start, eps_end, eps_decay, episode_train, and batch_size
        - The simulation environment
    - **Methods:** 
        - `create_directory()` - for storing the training results for each hyperparameter configuration
        - `eps_greedy_policy()` - 


In [26]:
import random                       # To draw a random batch of samples form the replay buffer
import gymnasium as gym
import torch
from torch import nn
import torch.optim as optim
from torchsummary import summary

import numpy as np
import matplotlib.pyplot as plt

import os                                               # For saving models and training results
from datetime import datetime                           # For creating the directory of each training run
import json                                             # For storing training parameters during each run
import re                                               # For checking the latest trial #
from contextlib import redirect_stdout

from collections import deque       # replay buffer is a double ended queue that allows elements to be added either to the end or the start

# Setup

In [2]:
env = gym.make('CartPole-v1')
obs,_ = env.reset()

obs_space = env.observation_space.shape[0]
action_space = env.action_space.n

In [3]:
class QNet_MLP(nn.Module):
    def __init__(self,input_dim,output_dim,hidden_layer: list):
        super().__init__()

        self.layers = nn.ModuleList()
        self.input_dim = input_dim
        self.output_dim = output_dim

        for size in hidden_layer:
            self.layers.append(nn.Linear(self.input_dim, size))
            self.input_dim = size
        
        self.layers.append(nn.Linear(self.input_dim,self.output_dim))
    
    def forward(self, input_data):
        for layer in self.layers:
            input_data = layer(input_data)
        return input_data 

In [ ]:
def create_directory(trial_number: int,
                     model_id: int,
                     lr: float, 
                     gamma: float,
                     epsilon_decay: int,
                     batch_size: int, 
                     buffer_size: int,
                     target_update_freq: int):
    ''' Function that creates directory to save model state_dict, architecture, training configurationi and history

    Parameters: 
    ------------
    (hyperparameters for differentiating between different directory)
    
    lr : float
        the learning rate to optimize the Q network
    gamma : float 
        the discount rate in Q learning
    epsilon_decay : integer
        the amount of episode over which the exploratory rate (epsilon) decays
    batch_size : integer
        number of experience drawn from replay buffer to train the behaviour network
    buffer_size : integer
        the number of samples in the replay buffer at a time
    target_udpate_freq : integer
        the amount of step count during the training process before updating the target Q net (loading the parameters of the behaviour net onto the target Q Net)
    

    Returns
    ------------
    name_codified : str
        the shortened name for the current experiment 
    OUTPUT_DIR : path
        the directory to which the trainin results and model (state_dict and architecture) are saved

    '''
    
    timestamp = datetime.now().strftime("%y%m%d_%H%M")
    BASE_DIR = os.getcwd()

    name_codified = f"run_{trial_number:04d}"
    hyperparam_codified = f"{timestamp}_{model_id}_{lr}_{gamma}_{epsilon_decay}_{batch_size}_{buffer_size}_{target_update_freq}"

    OUTPUT_DIR = os.path.join(BASE_DIR,"inv_pend_results",name_codified)
    os.makedirs(OUTPUT_DIR, exist_ok=True)      # Create the directory

    # Store the training configs in JSON file
    training_params = {
        'lr': lr,
        'gamma': gamma,
        'epsilon_decay': epsilon_decay,
        'batch_size': batch_size,
        'buffer_size': buffer_size,
        'target_update_freq': target_update_freq
    }

    params_path = os.path.join(OUTPUT_DIR, "params_results.json")
    with open(params_path, "w") as f:
        json.dump({"parameters": training_params}, f, indent=2)


    return name_codified, hyperparam_codified, OUTPUT_DIR

In [ ]:
def eps_greedy_policy(env, obs, epsilon: float, q_network: QNet_MLP):
    ''' Function to take an action according to an epsilon-greedy policy and a Q-network'''
    if np.random.random() < epsilon:
        action = env.action_space.sample()
    else:
        with torch.no_grad():
            state_tensor = torch.FloatTensor(obs).unsqueeze(0)
            q_values = q_network(state_tensor)
            action = q_values.argmax().item()

    return action

In [ ]:
def DQN_train(q_network: QNet_MLP, 
              target_net: QNet_MLP, 
              optimizer, 
              replay_buffer,
              target_update_freq,
              gamma,
              eps_start,
              eps_end,
              eps_decay,
              episode_train,
              batch_size):
    ''' Function to train a policy for a set of hyperparameters '''

    reward_history = np.zeros(episode_train)
    epsilon = eps_start
    step_count = 0
    episode = 0
    
    train_terminated = False
    
    while not train_terminated:     # Experiment level - loop through episodes
        episode += 1

        obs, _ = env.reset()
        eps_rewards = 0

        while True:                 # Episode level - loop through steps
            action = eps_greedy_policy(env, obs, epsilon, q_network)

            next_obs, reward, term, trunc, _ = env.step(action)
            done = term or trunc
            replay_buffer.append((obs, action, reward, next_obs, done))
            obs = next_obs
            eps_rewards += reward
            step_count += 1

            # Sample a batch and train the network
            if len(replay_buffer) >= batch_size:
                batch = random.sample(replay_buffer, batch_size)
                states, actions, rewards, next_states, dones = zip(*batch)

                states = torch.FloatTensor(states)                      # Shape of (BATCH_SIZE, 4). Data structure of [[p1 v1 theta1 dtheta1], [p2 v2 theta2 dtheta2], ...]
                actions = torch.LongTensor(actions).unsqueeze(1)        # Must unsqueeze to have shape (BATCH_SIZE, 1). From [a1, a2, ...] to [[a1], [a2], ... [a_BATCH_SIZE]]
                rewards = torch.FloatTensor(rewards).unsqueeze(1)
                next_states = torch.FloatTensor(next_states)
                dones = torch.FloatTensor(dones).unsqueeze(1)
                
                # Compute targets using target network Q(s',a',w_i^-)
                with torch.no_grad():
                    target_q_values = target_net(next_states)       # Find a batch of Q(s',a',w_i^-) from the batch of next_states
                    max_target_q_values = target_q_values.max(dim=1, keepdim=True)[0]
                    targets = rewards + gamma * max_target_q_values * (1 - dones)       # (1-dones) to avoid over estimating the value of terminal states. The target should only be the reward in the terminal states
            
                # Compute the current Q values for the actions taken Q(s,a,w_i)
                q_values = q_network(states).gather(1, actions)         # obtain the q_values associated to the actual action taken in each sample

                # Update the parameters of the behaviour q_network
                loss = nn.MSELoss()(q_values, targets)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Periodically update the target network by loading the weights from the behavior network
                if step_count % target_update_freq == 0:
                    target_network_update_count += 1
                    target_net.load_state_dict(q_network.state_dict())

            if done:        # End of a training episode
                break
        
        # TO DO: Monitor the performance during training for signs of good performance
        
        # Checking for early training termination or truncation
        train_terminated = (episode < episode_train) or (performance_crit)



In [ ]:
# Test creating the Q network with dynamic hidden layer when specifying the list of hidden nodes
%%script test --no-raise-error
Q_net = QNet_MLP(obs_space,action_space,[64,32])
summary(Q_net, (obs_space,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             320
            Linear-2                   [-1, 32]           2,080
            Linear-3                    [-1, 2]              66
Total params: 2,466
Trainable params: 2,466
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


In [6]:
%%script test --no-raise-error
BASE_DIR = os.getcwd()
RESULTS_DIR = os.path.join(BASE_DIR,"inv_pend_results")
os.makedirs(RESULTS_DIR, exist_ok = True)

existing_runs = [d for d in os.listdir(RESULTS_DIR) if os.path.isdir(os.path.join(RESULTS_DIR,d))]
existing_runs

['250520_165708_5000_32_0.001_0.95_1000_5000', 'run_0001']

# Parameter Set

In [7]:
model_registry = {
    'DQN_MLP_v0': {
        'class': QNet_MLP,
        'config': [64,32]
    },
    'DQN_MLP_v1': {
        'class': QNet_MLP,
        'config': [32,32]
    },
    'DQN_MLP_v2': {
        
        'class': QNet_MLP,
        'config': [32,16]
    },
    'DQN_MLP_v3': {
        'class': QNet_MLP,
        'config': [16,16]
    }
}

The block below is used for testing the model creation automation (Uncomment the first line to run the test)

In [9]:
%%script test --no-raise-error
for model in model_registry:
    Q_net = QNet_MLP(obs_space, action_space, model_registry[model]['config'])
    # print(model_registry[model]['config'])
    print(Q_net)

Couldn't find program: 'test'


In [31]:
param_grid = {
    'MODEL': [model for model in model_registry],
    'LR': [5e-4, 1e-3, 5e-3, 1e-2],
    "BUFFER_SIZE": [1e3, 5e3, 1e4],
    "MIN_REPLAY_SIZE": [1000],
    "TARGET_UPDATE_FREQ": [1e3, 5e3, 1e4],

    "GAMMA": [0.90, 0.95, 0.98],
    "EPSILON_START": [1.0],
    "EPSILON_END": [0.1],
    "EPSILON_DECAY": [1000, 5000, 10000],

    "EPISODE_TRAIN": [5000],                # training truncation criteria
    "BATCH_SIZE": [32, 64, 128]
}

success_criteria = 450                      #


In [36]:
model_list = [model for model in model_registry]
model_list[1]

'DQN_MLP_v1'

# Grid Search

Using itertools to loop through each combination of hyperparameters

In [ ]:
import itertools

# keys, values = zip(*param_grid.items())
keys, values = param_grid.keys(), param_grid.values()
for v in itertools.product(*values):
    # Initialize the environment
    env = gym.make('CartPole-v1')
    obs,_ = env.reset()

    obs_space = env.observation_space.shape[0]
    action_space = env.action_space.n

    # Unpacking the hyperparameter configurations
    config = dict(zip(keys,v))
    MODEL_CONFIG = model_registry[config['MODEL']]['config']
    LR = config['LR']
    BUFFER_SIZE = config['BUFFER_SIZE']
    MIN_REPLAY_SIZE = config['MIN_REPLAY_SIZE']
    TARGET_UPDATE_FREQ = config[TARGET_UPDATE_FREQ]
    GAMMA = config[GAMMA]
    EPSILON_START = config[EPSILON_START]
    EPSILON_END = config[EPSILON_END]
    EPSILON_DECAY = config[EPSILON_DECAY]
    EPISODE_TRAIN = config[EPISODE_TRAIN]
    BATCH_SIZE = config[BATCH_SIZE]

    # Initialize the NN models
    Q_net = QNet_MLP(obs_space,action_space,MODEL_CONFIG)
    target_net = QNet_MLP(obs_space,action_space,MODEL_CONFIG)
    target_net.load_state_dict(Q_net.state_dict())
    target_net.eval()

    optimizer = optim.SGD(Q_net.parameters(), lr = LR)

    # Initialize the replay buffer
    replay_buffer = deque(maxlen = BUFFER_SIZE)

    # Train the DQN with the given hyperparameter configuration
    DQN_train(Q_net,target_net,optimizer,replay_buffer,
              TARGET_UPDATE_FREQ,
              GAMMA,
              EPSILON_START,
              EPSILON_END,
              EPSILON_DECAY,
              EPISODE_TRAIN,
              BATCH_SIZE)

1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0
5000.0

In [ ]:
trial_number = 0
trial_dict = {}

for model_number, model in enumerate(model_registry):
    hidden_layers = model_registry[model]['config']
    QNet = QNet_MLP(obs_space, action_space, hidden_layers)

    for lr in LR:
        for gamma in GAMMA:
            for epsilon_decay in EPSILON_DECAY:
                for batch_size in BATCH_SIZE:
                    for buffer_size in BUFFER_SIZE:
                        for target_update_freq in TARGET_UPDATE_FREQ:
                            # Create directory to store the training configuration and results
                            trial_number += 1
                            name, hyperparam, path = create_directory(trial_number,model_number+1,lr,gamma,epsilon_decay,batch_size,buffer_size,target_update_freq)
                            trial_dict[name] = hyperparam
                            print(hyperparam)

                            model_path = os.path.join(path,'model_summary.json')
                            with open(model_path, 'w') as f:
                                with redirect_stdout(f):
                                    summary(QNet, (obs_space, ))

                            # Keep track of the best performance and weights of this best network.
                            break
                        break
                    break
                break
            break
        break
    break                      
                            # Store the training parameters

                            # Plot the result and store in the directory
                            # for episode in range(max(EPISODE_TRAIN)):   # Train to the max number of episode_train and save checkpoints along the way

250521_1516_1_0.0005_0.9_1000_32_1000.0_1000.0


In [17]:
QNet = QNet_MLP(obs_space, action_space, [64, 32])
summary(QNet, (obs_space,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             320
            Linear-2                   [-1, 32]           2,080
            Linear-3                    [-1, 2]              66
Total params: 2,466
Trainable params: 2,466
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.01
----------------------------------------------------------------
